# 2_train_model.ipynb

In [158]:
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Input
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import concatenate
from keras.utils import plot_model
from keras import regularizers
from keras import losses
from keras import callbacks
from keras import metrics
import keras
import pickle
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve

## Load Data

In [166]:
# load data
with open('data/all_data.pkl', 'rb') as file:
    price_series, time_categories, up_targets, dn_targets = pickle.load(file) 

timesteps = price_series.shape[1]
channels = price_series.shape[2]
time_dim = time_categories.shape[1]

## Split Data

In [167]:
RANDOM_SEED = 7
TARGETS = up_targets

# Splitting the arrays into train and test sets
train_x_price, test_x_price = train_test_split(price_series, test_size=0.2, random_state=RANDOM_SEED)
train_x_time, test_x_time = train_test_split(time_categories, test_size=0.2, random_state=RANDOM_SEED)
train_y, test_y = train_test_split(TARGETS, test_size=0.2, random_state=RANDOM_SEED)

## Define Multimodal Convolutional Network

In [ ]:
FILTERS = 256

keras.utils.set_random_seed(7)

# convolutional branch
input_cnn = Input(shape=(timesteps,channels)) 

cnn = Conv1D(filters=FILTERS, kernel_size=7, padding='same', 
             activation='relu', data_format='channels_last',
             activity_regularizer=None,
             input_shape=(timesteps, channels))(input_cnn)

cnn = Conv1D(filters=FILTERS, kernel_size=3, padding='same', 
             activation='relu', data_format='channels_last', 
             activity_regularizer=None)(cnn)

cnn = Conv1D(filters=FILTERS, kernel_size=2, padding='same', strides=2,      # pooling layer
             activation='relu', use_bias=False, data_format='channels_last', 
             activity_regularizer=None)(cnn)

cnn = Conv1D(filters=FILTERS*2, kernel_size=3, padding='same', 
             activation='relu', data_format='channels_last', 
             activity_regularizer=None)(cnn)

cnn = Conv1D(filters=FILTERS*2, kernel_size=2, padding='same', strides=2,     # pooling layer
             activation='relu', use_bias=False, data_format='channels_last',
             activity_regularizer=None)(cnn)

cnn = Conv1D(filters=FILTERS*4, kernel_size=3, padding='same', 
             activation='relu', data_format='channels_last', 
             activity_regularizer=regularizers.L2(0.01))(cnn)

cnn = Dropout(0.4)(cnn)
cnn = Flatten()(cnn)
cnn = Model(inputs=input_cnn, outputs=cnn)

# perceptron branch
input_mlp = Input(shape=(time_dim,))
mlp = Dense(8, activation='relu', 
            activity_regularizer=regularizers.L2(0.01))(input_mlp)
mlp = Model(inputs=input_mlp, outputs=mlp)

# join branches
combined = concatenate([cnn.output, mlp.output])
head = Dense(1024, activation='relu')(combined)
head = Dense(512, activation='relu')(head)
head = Dense(2, activation='softmax')(head)
model = Model(inputs=[cnn.input, mlp.input], outputs=head)

model.compile(loss='binary_focal_crossentropy', 
              optimizer='adam', 
              metrics=['binary_accuracy'])

print(model.summary())

plot_model(model, to_file='model_plot.png', 
                  show_shapes=True, 
                  show_layer_names=True, 
                  show_layer_activations=True)

## Train And Evaluate Model

In [ ]:
BATCH_SIZE = 512

callback = keras.callbacks.EarlyStopping(monitor="val_loss",
   				 					 	 patience=2,
    			 					  	 verbose=0,
				 						 restore_best_weights=True,)

model.fit(x=[train_x_price, train_x_time], y=train_y,
	      validation_data=([test_x_price, test_x_time], test_y),
	      epochs=20, batch_size=BATCH_SIZE, callbacks=callback)

loss, accuracy = model.evaluate([test_x_price, test_x_time], test_y, batch_size=BATCH_SIZE, verbose=0)
print(loss, accuracy)

model.save('models/up/cnn3.keras')

## Evaluate Precision-Recall Curves

In [ ]:
model = keras.models.load_model('models/up/cnn.keras')

labels = model.predict([test_x_price, test_x_time])
# keep probabilities for the positive outcome only
probs = labels[:, 1]
print('probs:',probs)

print('labels:',labels)
y_true = test_y[:, 0]
print('test_y',test_y)   ### test_y allows [0. 0.] elements - fix first!
print('y_true:',y_true)

precision, recall, thresholds = precision_recall_curve(y_true, probs)

# plot the precision-recall curves
no_skill = len(test_y[test_y==1]) / len(test_y)
plt.plot([0, 1], [no_skill, no_skill], linestyle='--', label='No Skill')
plt.plot(recall, precision, marker='.', label='CNN_up')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.legend()
plt.show()

### Notes

In [ ]:
## add cross validation
## pad - most recent edge is important! #
## replace maxpooling with dilated kernel to preserve data - geoff hinton #
## replace maxpooling with stride = 2 - 'learnable pooling, all-convolution network' #
## batch norm on conv layers? #
## groups = channels maintains depthwise separation.  Add same multiple of filters?
## l2 less sensitive to small changes (noise), l1 blocks some inputs #
## layers.DepthwiseConv1D() reduces number of parameters
## model.train(sample_weight=) heavier recent samples